# Analyse de données sur une histoire du conflit politique

On va d'abord importer et nettoyer les données

Voici le code que j'ai fait pour transformer un csv de 700Mo en un fichier parquet plus léger qui ne contient que les colonnes nécessaires, pour le commit sur git

In [9]:
import polars as pl

# On prépare la requête sans charger les données
overrides = {
    "codecommune": pl.Categorical,
    "dep": pl.Categorical    
}

query = (
    pl.scan_csv("leg2022comm.csv",
        schema_overrides=overrides, 
        separator=",")
    .drop([
        "nomdep",
        "nomcommune",
        "inscrits",
        "votants",
        'exprimes',
        'plm',
        'plmdoublon',
        'electeurs'
        ])
)

df = query.collect()

print(df.head())
# Sauvegarde en Parquet (nécessite pyarrow, déjà installé avec polars souvent)
df.write_parquet("data_vote2022.parquet")

shape: (5, 80)
┌─────┬─────────────┬─────────┬─────────┬───┬──────────┬───────────┬──────────┬─────────────┐
│ dep ┆ codecommune ┆ voixAUG ┆ voixNUP ┆ … ┆ pins     ┆ pinsratio ┆ pabs     ┆ pblancsnuls │
│ --- ┆ ---         ┆ ---     ┆ ---     ┆   ┆ ---      ┆ ---       ┆ ---      ┆ ---         │
│ cat ┆ cat         ┆ i64     ┆ i64     ┆   ┆ f64      ┆ f64       ┆ f64      ┆ f64         │
╞═════╪═════════════╪═════════╪═════════╪═══╪══════════╪═══════════╪══════════╪═════════════╡
│ 01  ┆ 01001       ┆ 4       ┆ 52      ┆ … ┆ 1.0      ┆ 1.0759655 ┆ 0.467391 ┆ 0.006211    │
│ 01  ┆ 01002       ┆ 4       ┆ 44      ┆ … ┆ 1.0      ┆ 1.0759655 ┆ 0.389908 ┆ 0.022936    │
│ 01  ┆ 01004       ┆ 66      ┆ 1196    ┆ … ┆ 0.894426 ┆ 0.962372  ┆ 0.533808 ┆ 0.012551    │
│ 01  ┆ 01005       ┆ 5       ┆ 100     ┆ … ┆ 0.84823  ┆ 0.912666  ┆ 0.510393 ┆ 0.010778    │
│ 01  ┆ 01006       ┆ 3       ┆ 13      ┆ … ┆ 1.0      ┆ 1.0759655 ┆ 0.376238 ┆ 0.039604    │
└─────┴─────────────┴─────────┴─────────┴───┴